# Practical Part

Implement a small app that uses all concepts introduced in the theory part.

Ideas:

    - Fusion-Chef: Combines two dishes into something new - also knows the fitting cocktail to everything. Saves delicious sounding recipes in the memory.
    - QnA Bot: Searches a prefilled memory for answers to certain questions
    - Roleplay Gamemaster: Generates roleplay encounters and keeps track of the party's items & stats
    - Shell Meister: Can translate natural language into shell commands and executes them (only recommended when using a devcontainer). Saves the results in memory if the system is still working....

### Let's set up the Kernal

In [17]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import KernelArguments
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

settings = chat_service.get_prompt_execution_settings_class()
settings.temperature = 0.7
settings.top_p = 0.8

kernel.add_service(chat_service)

# Test the kernel
print("Kernel initialized successfully!")

Kernel initialized successfully!


### Let's create a FusionChef plugin

In [18]:
class FusionChef:
    def __init__(self, kernel: Kernel):
        template_dish = """
        Dish1: {{$dish1}}
        Dish2: {{$dish2}}
        
        Combines these two dishes into something new and give me a recipe.
        """

        template_cocktail = """
        dish: {{$dish}}
        
        Give me a cocktail recipe that goes well with this dish.
        """


        self.dish = kernel.add_function(
            plugin_name="FusionChef",
            function_name="CombineDishes",
            prompt=template_dish,
            template_format="semantic-kernel"
        )

        self.cocktail = kernel.add_function(
            plugin_name="FusionChef",
            function_name="CombineCocktails",
            prompt=template_cocktail,
            template_format="semantic-kernel"
        )

        memory_store = VolatileMemoryStore()
        self.memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
        kernel.add_plugin(TextMemoryPlugin(self.memory), "TextMemoryPlugin")
        self.id = 0

    async def combine_dishes(self, dish1: str, dish2: str):
        return await kernel.invoke(
            self.dish,
            KernelArguments(dish1=dish1, dish2=dish2)
        )
    
    async def combine_cocktail(self, dish: str):
        return await kernel.invoke(
            self.cocktail,
            KernelArguments(dish=dish)
        )
    
    async def save_memory(self, collection: str, text: str, description: str):
        self.id += 1
        await self.memory.save_information(
            collection=collection,
            id = str(self.id),
            text=str(text),
            description=description
            )

chef = FusionChef(kernel)

# Encode a story
recepie = await chef.combine_dishes("pumpkin soup", "brownies")
print("\nRecepie: \n", recepie)




Recepie: 
 Sure! Let's create a delightful dessert called **Pumpkin Brownie Swirl**. This recipe combines the rich, fudgy texture of brownies with the warm, comforting flavors of pumpkin soup, resulting in a deliciously unique treat.

### Pumpkin Brownie Swirl Recipe

#### Ingredients:

**For the Brownie Base:**
- 1/2 cup (1 stick) unsalted butter
- 1 cup granulated sugar
- 2 large eggs
- 1 teaspoon vanilla extract
- 1/3 cup unsweetened cocoa powder
- 1/2 cup all-purpose flour
- 1/4 teaspoon salt
- 1/4 teaspoon baking powder

**For the Pumpkin Swirl:**
- 3/4 cup canned pumpkin puree (not pumpkin pie filling)
- 1/4 cup granulated sugar
- 1/2 teaspoon cinnamon
- 1/4 teaspoon nutmeg
- 1/4 teaspoon vanilla extract
- 1 egg

#### Instructions:

1. **Preheat the Oven:**
   Preheat your oven to 350°F (175°C) and grease an 8x8-inch baking pan or line it with parchment paper for easy removal.

2. **Make the Brownie Base:**
   - In a medium saucepan, melt the butter over low heat. Remove from he

In [19]:
cocktail = await chef.combine_cocktail("pumpkin soup")
print("\nCocktail recepie: \n", cocktail)



Cocktail recepie: 
 A great cocktail to pair with pumpkin soup is a spiced apple cider cocktail. The warm, autumnal flavors complement the soup beautifully. Here’s a simple recipe:

### Spiced Apple Cider Cocktail

#### Ingredients:
- 4 oz apple cider
- 1 oz bourbon (or rum for a sweeter variant)
- 1/2 oz fresh lemon juice
- 1/2 oz maple syrup (optional, for added sweetness)
- Cinnamon stick (for garnish)
- Apple slices (for garnish)
- Ice (optional)

#### Instructions:

1. **Mix Ingredients**: In a shaker, combine the apple cider, bourbon, lemon juice, and maple syrup if using.
2. **Shake or Stir**: If you prefer it chilled, shake with ice. If you prefer it warm, simply stir together.
3. **Serve**: Pour into a glass or mug. For a warm drink, heat the apple cider in a saucepan before mixing.
4. **Garnish**: Add a cinnamon stick and a few apple slices for garnish. 

#### Notes:
- For a non-alcoholic version, simply omit the bourbon and use spiced apple cider instead.
- You can adjust t

In [20]:
await chef.save_memory("dishes", str(recepie), "A delicious soup made from pumpkin")
result = await chef.memory.search("dishes", "What was the recipe for the pumpkin soup?")

print("\nMemorized result: \n", result[0].text)


Memorized result: 
 Sure! Let's create a delightful dessert called **Pumpkin Brownie Swirl**. This recipe combines the rich, fudgy texture of brownies with the warm, comforting flavors of pumpkin soup, resulting in a deliciously unique treat.

### Pumpkin Brownie Swirl Recipe

#### Ingredients:

**For the Brownie Base:**
- 1/2 cup (1 stick) unsalted butter
- 1 cup granulated sugar
- 2 large eggs
- 1 teaspoon vanilla extract
- 1/3 cup unsweetened cocoa powder
- 1/2 cup all-purpose flour
- 1/4 teaspoon salt
- 1/4 teaspoon baking powder

**For the Pumpkin Swirl:**
- 3/4 cup canned pumpkin puree (not pumpkin pie filling)
- 1/4 cup granulated sugar
- 1/2 teaspoon cinnamon
- 1/4 teaspoon nutmeg
- 1/4 teaspoon vanilla extract
- 1 egg

#### Instructions:

1. **Preheat the Oven:**
   Preheat your oven to 350°F (175°C) and grease an 8x8-inch baking pan or line it with parchment paper for easy removal.

2. **Make the Brownie Base:**
   - In a medium saucepan, melt the butter over low heat. Remov